# Fx Forward Pricing

This notebook will run through the following business use cases :
* [Pricing a set of Fx Forwards using a the internal LUSID "Discounting" model](#pricing_bond)
* [Valuing a portfolio made up of an Fx forward position.](#pricing_bond_portfolio)
* [Valuing our Fx forward portfolios using an externally supplied Fx forward price.](#accrual_override)

<br>

In doing so we'll cover the following LUSID concepts :
* [Defining a LUSID internal representation of an FX forward instrument based on user provided parameters.](#bond_definition)
* [Supplying both simple and complex market data structures through the Quotes and the StructureMarketData store.](#structured_market_data)
* [Configuring recipes to value our FX forward in LUSID making use of the structured market data (GBP and USD OIS Yield Curves).](#recipe_configuration)
* [Running inline aggregations that value a unitised FX forward.](#accrued_interest)
* Running portfolio aggregations that value a porfolio containing FX forwards.
* [Using an externally valued FX forward price to calculate portd.](#external_bond_price)

<br>

For this notebook example we'll work the following set of GBP/USD Forwards with 3Y, 5Y and 10Y maturities:

* Trade Date : 02 Aug 2020
* Maturity Dates :
    * 02 Aug 2023 (3Y)
    * 02 Aug 2025 (5Y)
    * 02 Aug 2030 (10Y)
* Spot Rate : 1.2508
* Domestic Amount : 100,000,000
* Foreign Amounts :
    * 125,710,000 (3Y)
    * 126,485,000 (5Y)
    * 130,430,000 (10Y)


## Setup LUSID and LUSID API objects.

In [82]:
import os
from datetime import datetime, timedelta

import lusid
import pandas as pd
import pytz
from IPython.core.display import display
from lusid import models
from lusid.utilities import ApiClientFactory
from lusidjam import RefreshingToken
from lusidtools.cocoon.transaction_type_upload import (
    create_transaction_type_configuration,
)
from lusidtools.cocoon.cocoon_printer import (
    format_portfolios_response,
)
from lusidtools.jupyter_tools.stop_execution import StopExecution


# Authenticate our user and create our API client
from lusidtools.cocoon import load_from_data_frame
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame
secrets_path = os.getenv("FBN_SECRETS_PATH")

# Initiate an API Factory which is the client side object for interacting with LUSID APIs
api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename=secrets_path,
    app_name="LusidJupyterNotebook",
)

print ('LUSID Environment Initialised')
print ('LUSID SDK Version: ', api_factory.build(lusid.api.ApplicationMetadataApi).get_lusid_versions().build_version)

# Setup the apis we'll use in this notebook:
instruments_api = api_factory.build(lusid.api.InstrumentsApi)
structured_market_data_api = api_factory.build(lusid.api.StructuredMarketDataApi)
structured_result_data_api = api_factory.build(lusid.api.StructuredResultDataApi)
quotes_api = api_factory.build(lusid.api.QuotesApi)
transaction_portfolios_api = api_factory.build(lusid.api.TransactionPortfoliosApi)
system_configuration_api = api_factory.build(lusid.api.SystemConfigurationApi)
# Setup the scope we'll use in this notebook:
scope = "fx-forward-pricing-nb"

LUSID Environment Initialised
LUSID SDK Version:  0.6.4767.0


In [2]:
# Settings and utility functions to display objects and responses more clearly.
pd.set_option('float_format', '{:f}'.format)
def aggregation_result_to_dataframe(aggregation_results):
    df = pd.DataFrame(aggregation_results, columns = ['Name', 'Effective At', 'Value'])
    df['Effective At'] = df['Effective At'].dt.strftime('%d %b %Y')
    return df

def display_holdings_summary(response, effective_at):
    # inspect holdings response for today
    hld = [i for i in response.values]

    names = []
    #amount = []
    units = []
    holding_types = []
    cost = []
    settled_units = []


    for item in hld:

        names.append(item.properties["Instrument/default/Name"].value.label_value)
        #amount.append(item.cost.amount)
        units.append(item.units)
        holding_types.append(item.holding_type)
        settled_units.append(item.settled_units)
        cost.append(item.cost)


    data = {"names": names, "effectiveAt" : effective_at, "units": units, "settled_units": settled_units, "holding types": holding_types, "cost" : cost}

    summary = pd.DataFrame(data=data)
    return summary

## Define our Fx Forward Instruments

We'll start by defining our forwards using LUSID's internal representation of an Fx forward instrument. For a detailed
breakdown the internal representation take a look at the "FxForwardInstrument" section of the [LUSID API Swagger Specification](https://www.lusid.com/api/swagger/index.html)

### Initialise FxForward Parameters

Let's setup the basic parameters required for our set of Fx forwards:

In [3]:
trade_date = datetime(2020, 6, 1, tzinfo=pytz.utc)
#trade_date = datetime.today().replace(tzinfo=pytz.utc)
maturity_dates = {
    '1W' : datetime(2020, 6, 8, tzinfo=pytz.utc),
    '3Y' : datetime(2023, 6, 1, tzinfo=pytz.utc),
    '5Y' : datetime(2025, 6, 1, tzinfo=pytz.utc),
    '10Y': datetime(2030, 6, 1, tzinfo=pytz.utc)
}
spot_rate =  1.2508
dom_amount = 100000000
fgn_amounts = {
    '1W' : 125090000,
    '3Y' : 125710000,
    '5Y' : 126485000,
    '10Y': 130430000
}

### Create FxForwardInstrument Definitions

We can use those parameters to construct a LUSID representation of an FxForward. In our case we'll define a set made up of a
3Y, 5Y and 10Y GBP/USD Fx Forward.

In [4]:
def create_fx_forward_instrument_definition(dom_amount, fgn_amount, spot_rate, start_date, maturity_date):
    return models.FxForwardInstrument(
            dom_amount=dom_amount,
            fgn_amount=-fgn_amount,
            fgn_ccy="USD",
            ref_spot_rate=spot_rate,
            start_date=start_date.isoformat(),
            maturity_date=maturity_date,
            dom_ccy="GBP",
            instrument_type="FxForward")

gbpusd_fx_fwd_1w_instr_def = create_fx_forward_instrument_definition(dom_amount, fgn_amounts['1W'], spot_rate, trade_date, maturity_dates['1W'])
gbpusd_fx_fwd_3y_instr_def = create_fx_forward_instrument_definition(dom_amount, fgn_amounts['3Y'], spot_rate, trade_date, maturity_dates['3Y'])
gbpusd_fx_fwd_5y_instr_def = create_fx_forward_instrument_definition(dom_amount, fgn_amounts['5Y'], spot_rate, trade_date, maturity_dates['5Y'])
gbpusd_fx_fwd_10y_instr_def = create_fx_forward_instrument_definition(dom_amount, fgn_amounts['10Y'], spot_rate, trade_date, maturity_dates['10Y'])

## Supply Market Data

Before we can price our Fx forwards we need to ensure LUSID has the required FX and interest rates available. This allows us
to showcase two ways of getting the market data we need to price an FX forward into LUSID. The simpler method for the FX rates
can simply be upserted as quotes but the more core complex structure of the FGPB and USD yield curves required the use of the
Structured Market Data store.

Let's start by upserting the GBP/USD FX rates for the trade data and a few subsequent dates we'll use to price the FX Forwards.
So we can see some movement in the forward prices we'll mock a drop in the spot rate on June 2nd followed by a sharp rise on June 3rd. When providing market
data to LUSID we also need to identify the supplier of the data and it's scope. The supplier is required further in the
notebook when we instruct LUSID how to source data for a particular aggregation.

### Upsert Spot Fx Rates

In [5]:
market_data_scope = 'fx-forward-pricing-nb-market-data'
market_supplier = 'Lusid'

trade_date_plus_one = datetime(2020, 6, 2, tzinfo=pytz.utc)
trade_date_plus_two = datetime(2020, 6, 3, tzinfo=pytz.utc)


def upsert_gbp_usd_fx_rate(rate, effective_at):
    upsert_quote_request = models.UpsertQuoteRequest(
        quote_id=models.QuoteId(
            quote_series_id=models.QuoteSeriesId(
                provider=market_supplier,
                instrument_id="GBP/USD",
                instrument_id_type='CurrencyPair',
                quote_type='Price',
                field='mid'),
            effective_at=effective_at),
        metric_value=models.MetricValue(
            value=rate,
            unit='rate'
        ),
        lineage='FxDataVendorABC')

    # we need to insert the USD/GBP quote as well for use later in the notebook
    # when we're booking the sell leg of our forward as a transaction
    upsert_quote_request_inverse = models.UpsertQuoteRequest(
        quote_id=models.QuoteId(
            quote_series_id=models.QuoteSeriesId(
                provider=market_supplier,
                instrument_id="USD/GBP",
                instrument_id_type='CurrencyPair',
                quote_type='Price',
                field='mid'),
            effective_at=effective_at),
        metric_value=models.MetricValue(
            value=1/rate,
            unit='rate'
        ),
        lineage='FxDataVendorABC')

    response_gbp_usd = quotes_api.upsert_quotes(
            scope=market_data_scope,
            request_body={"gbp-usd-01": upsert_quote_request})

    response_usd_gbp = quotes_api.upsert_quotes(
            scope=market_data_scope,
            request_body={"usd-gbp-01": upsert_quote_request_inverse})

    if response_gbp_usd.failed or response_usd_gbp.failed:
        raise StopExecution(f"Failed to upload currency pairs:{response_gbp_usd.failed} or {response_usd_gbp.failed}")

    display(f"GBP/USD @ {rate} for {effective_at} uploaded to quote store.")
    display(f"USD/GBP @ {1/rate} for {effective_at} uploaded to quote store.")


upsert_gbp_usd_fx_rate(spot_rate, trade_date)
# drop in fx rate
upsert_gbp_usd_fx_rate(spot_rate - 0.0080, trade_date_plus_one)
# increase in fx rate
upsert_gbp_usd_fx_rate(spot_rate + 0.0080, trade_date_plus_two)


'GBP/USD @ 1.2508 for 2020-06-01 00:00:00+00:00 uploaded to quote store.'

'USD/GBP @ 0.799488327470419 for 2020-06-01 00:00:00+00:00 uploaded to quote store.'

'GBP/USD @ 1.2428 for 2020-06-02 00:00:00+00:00 uploaded to quote store.'

'USD/GBP @ 0.804634695848085 for 2020-06-02 00:00:00+00:00 uploaded to quote store.'

'GBP/USD @ 1.2588 for 2020-06-03 00:00:00+00:00 uploaded to quote store.'

'USD/GBP @ 0.7944073721004131 for 2020-06-03 00:00:00+00:00 uploaded to quote store.'

### Interest Rate Curves

To price the FX Forwards we also need to supply the relevant interest rate curves for our domestic and foreign currencies. LUSID supports
storage of complex market data structures through the [Structured market data]("https://support.finbourne.com/how-do-i-store-and-tr")
store. When upserting the interest rate curves we use a StructuredMarketDataId to not onl uniquely identify the curve but also provide
the supporting information required by the aggregation engine to correctly resolve the curves when we run our FX forward valuation.

In [6]:
def upsert_interest_rate_curve(ir_curve_json, scope, effective_at, market_asset):

    # provide the structured data file source and it's document format
    structured_market_data = models.StructuredMarketData(document_format="Json", version="1.0.0",
                                                name="DFEUROISCurve", document=ir_curve_json)


    # create a unique identifier for our OIS yield curves
    structured_id = models.StructuredMarketDataId(provider="Lusid",price_source=None,
                                                 lineage="CurveVendorABC", effective_at=effective_at,
                                                 market_element_type="ZeroCurve",
                                                 market_asset=market_asset)

    upsert_request = models.UpsertStructuredMarketDataRequest(market_data_id=structured_id,
                                                          market_data=structured_market_data)

    # https://www.lusid.com/docs/api#operation/UpsertStructuredMarketData
    response = structured_market_data_api.upsert_structured_market_data(
        scope=scope,
        request_body={market_asset : upsert_request}
    )

    if response.failed:
        raise StopExecution("Failed to upload interest rates curve {response.failed}")

    print(f"{market_asset} interest rate curve uploaded into scope={scope}")


def load_ois_curve_json(ccy):
    with open(f"data/{ccy}OIS50.json", "r") as ir_curve_json:
        return ir_curve_json.read()

ois_curve_gbp_json = load_ois_curve_json('GBP')
upsert_interest_rate_curve(ois_curve_gbp_json, market_data_scope, trade_date, "GBP/GBPOIS")

ois_curve_usd_json = load_ois_curve_json('USD')
upsert_interest_rate_curve(ois_curve_usd_json, market_data_scope, trade_date, "USD/USDOIS")

GBP/GBPOIS interest rate curve uploaded into scope=fx-forward-pricing-nb-market-data
USD/USDOIS interest rate curve uploaded into scope=fx-forward-pricing-nb-market-data


## Defining our Fx Forward Valuation

With our Fx forward instruments now defined we can move onto valuing them via LUSID's aggregation engine. But before we
can execute the aggregation we need to configure a [Recipe]("https://support.finbourne.com/what-is-a-lusid-recipe-and-how-is-it-used")
instructing LUSID on how to value our Fx forwards. Specifically we need to select the Fx Forward pricing model to use and identify
where the aggregation can resolve the market data the model needs.

### Selecting a Pricing Model

The pricing model we wish to use is passed in through defining a PricingContext. See the [Swagger spec]("https://www.lusid.com/api/swagger/index.html") under "PricingContext" for a detailed
description of the parameters. For this notebook we'll use the LUSID supplied "Discounting" model:

In [7]:
def create_pricing_context():
    vendor_model_rule = models.VendorModelRule(
        supplier="Lusid",
        model_name="Discounting",
        instrument_type="FxForward",
        parameters="{}")

    return models.PricingContext(
        model_rules=[vendor_model_rule]
    )

pricing_context = create_pricing_context()

### Selecting the Market Data

We can instruct LUSID on where to resolve market data required for pricing our Fx forwards through defining a MarketContext. See the [Swagger spec]("https://www.lusid.com/api/swagger/index.html#model-MarketContext")
under "MarketContext" for a detailed description of the parameters. Recall that when we upserted our quote we passed in a supplier and a scope. One of
the powerful features in LUSID is the ability to run valuations against market data from different suppliers, or even define market data retrieval rules
to fallback to suppliers should your primary supplier not have the required data.

However our example is trivial as we only require the spot rate we loaded in earlier. So we'll define a simple market context :

In [8]:
def create_market_context():
    return models.MarketContext(
        # set rules for where we should resolve our rates data. In our case the interest rate curves we inserted into
        # the structured market result data store.
        market_rules=[
            models.MarketDataKeyRule(
                key="Rates.*.*",
                data_scope=market_data_scope,
                supplier=market_supplier,
                quote_type='Rate',
                field='Mid')
        ],
        # control default options for resolving market data. In our case simply default to the LUSID market_supplier
        # and market data scope we defined earlier.
        options=models.MarketOptions(
            default_supplier=market_supplier,
            default_scope=market_data_scope)
    )

    return market_context

market_context = create_market_context()

### Configure our FX Forward Recipe

Now that we've defined what pricing model to use and where to source the market data we can bring those instructions
together in a [Recipe]("https://support.finbourne.com/what-is-a-lusid-recipe-and-how-is-it-used"):

In [9]:
def create_fx_forward_pricing_recipe():
    return models.ConfigurationRecipe(
            scope=scope,
            code="fx-forward-pricing-recipe",
            description="Price Fx Forwards using LUSID internal model",
            market=market_context,
            pricing=pricing_context
        )

fx_forward_pricing_recipe = create_fx_forward_pricing_recipe()

## Price the FX Forwards

Let's summarise our current state:
 * We've defined a set of GBP/USD FX forwards for maturity in 3Y, 5Y and 10Y.
 * We've loaded in the spot GBP/USD rates for the dates we're going to value our forwards against.
 * We've setup our Recipe that configures how we would like to price our bond and where to source our required market data.

As we have no existing positions in the FX forward booked against a portfolio we'll run our aggregation on an inlined portfolio.
That is a portfolio made up of a set of weighted instruments only. In our case each request will be made up of only one instrument which
will be the FX forward we're valuing.

### Run an Aggregation to Price our Forwards

In [10]:
def run_fx_forward_pricing_aggregation(maturity, effective_at, fx_forward_instrument_definition, pricing_recipe):
    # setup weighted instrument (only our gilt definition)
    fx_forward = models.WeightedInstrument(quantity=1, instrument=fx_forward_instrument_definition, holding_identifier=f"{maturity}-holding")

    # create our aggregation request made up of our recipe and the metrics we would like to calculate (Bond PV)
    aggregation_request = models.AggregationRequest(
        effective_at=effective_at,
        inline_recipe=pricing_recipe,
        metrics=[
            models.AggregateSpec(key='Holding/default/PV', op='Value'),
        ]
    )

    # As we're running an inline aggregation we must wrap our original aggregation request with an inline aggregation
    # request and pass in our weighted instruments
    inline_aggregation_request = models.InlineAggregationRequest(
        request=aggregation_request, instruments=[fx_forward]
    )

    # https://www.lusid.com/docs/api#operation/GetAggregationOfWeightedInstruments
    return api_factory.build(lusid.api.AggregationApi).get_aggregation_of_weighted_instruments(
        market_data_scope, inline_aggregation_request=inline_aggregation_request)


Let's start by running the 5Y forward valuation across the three dates for which we upserted spot GBP/USD quotes. Recall the quotes were 1.2508 for
trade date, whic hthen dropped by 0.0080 on day two and subsequently then increased by 0.0160 on day 3.

In [11]:
result_t = run_fx_forward_pricing_aggregation('5Y', trade_date, gbpusd_fx_fwd_5y_instr_def, fx_forward_pricing_recipe)
result_t_plus_one = run_fx_forward_pricing_aggregation('5Y', trade_date_plus_one, gbpusd_fx_fwd_5y_instr_def, fx_forward_pricing_recipe)
result_t_plus_two = run_fx_forward_pricing_aggregation('5Y', trade_date_plus_two, gbpusd_fx_fwd_5y_instr_def, fx_forward_pricing_recipe)

aggregation_result_to_dataframe([
    ['GBP/USD 5Y Forward PV', trade_date, result_t.data[0]['Holding/default/PV']],
    ['GBP/USD 5Y Forward PV', trade_date_plus_one, result_t_plus_one.data[0]['Holding/default/PV']],
    ['GBP/USD 5Y Forward PV', trade_date_plus_two, result_t_plus_two.data[0]['Holding/default/PV']]
])

,Name,Effective At,Value
0,GBP/USD 5Y Forward PV,01 Jun 2020,-0.011177
1,GBP/USD 5Y Forward PV,02 Jun 2020,-0.017654
2,GBP/USD 5Y Forward PV,03 Jun 2020,-0.004782


Let's now take a slightly different perspective and look at the prices across the Forward curve for one date only:

In [12]:
result_3y = run_fx_forward_pricing_aggregation('3Y', trade_date, gbpusd_fx_fwd_3y_instr_def, fx_forward_pricing_recipe)
result_5y = run_fx_forward_pricing_aggregation('5Y', trade_date, gbpusd_fx_fwd_5y_instr_def, fx_forward_pricing_recipe)
result_10y = run_fx_forward_pricing_aggregation('10Y', trade_date, gbpusd_fx_fwd_10y_instr_def, fx_forward_pricing_recipe)

aggregation_result_to_dataframe([
    ['GBP/USD 3Y Forward PV', trade_date, result_3y.data[0]['Holding/default/PV']],
    ['GBP/USD 5Y Forward PV', trade_date, result_5y.data[0]['Holding/default/PV']],
    ['GBP/USD 10Y Forward PV', trade_date, result_10y.data[0]['Holding/default/PV']]
])


,Name,Effective At,Value
0,GBP/USD 3Y Forward PV,01 Jun 2020,-0.005022
1,GBP/USD 5Y Forward PV,01 Jun 2020,-0.011177
2,GBP/USD 10Y Forward PV,01 Jun 2020,-0.042347


## Pricing a Portfolio of FX Forwards

Up until now we've only priced a unit Fx Forward as an instrument. Let's now take it a step further and build a simple portfolio made up of
an FX forward positions. While we'll now be running our aggregation on an actual portfolio as opposed to just the Fx Forward instrument
we'll still make use of the same recipe we defined. This is because the rules regarding what model we'd like to use to price our
Fx Forward and where LUSID should retrieve market data from have not changed.


### Setting up our Portfolio

We'll firstly setup the portfolio that will hold our FX forward positions:

In [13]:
portfolio = "simple-fxfwd-portfolio-01"

def create_portfolio(scope, portfolio_code, portfolio_name, portfolio_ccy):
    pfs = [[portfolio_code, portfolio_name, portfolio_ccy]]
    pf_df = pd.DataFrame(pfs, columns=['portfolio_code', 'portfolio_name', 'base_currency'])

    portfolio_mapping = {
        "required": {
            "code": "portfolio_code",
            "display_name": "portfolio_name",
            "base_currency": "base_currency",
        },
        "optional": {"created": "$2020-01-01T00:00:00+00:00"},
    }
    result = load_from_data_frame(
        api_factory=api_factory,
        scope=scope,
        data_frame=pf_df,
        mapping_required=portfolio_mapping["required"],
        mapping_optional=portfolio_mapping["optional"],
        file_type="portfolios",
        sub_holding_keys=[],
    )
    succ, failed = format_portfolios_response(result)

    if not failed.empty:
        raise StopExecution(failed)

    return succ

create_portfolio(scope, portfolio, portfolio, "GBP")



,successful items
0,simple-fxfwd-portfolio-01


### Upserting our FX Forward Instrument

Previously as we were only pricing the FX Forward instrument we were only required to pass in it's instrument definition. As we're now
booking transactions against the instrument we'll need to upsert the instrument into LUSID. This will provide us with an identifier
we can use to book our transactions against.

For the purpose of our portfolio we'll only work with the 3Y FX forward from now on:

In [14]:
def upsert_fx_fwd_instrument(instrument_id, instrument_name, fx_fwd_definition):
    bond_instrument_request = {instrument_id: models.LusidInstrumentDefinition(
        # instrument display name
        name=instrument_name,
        # unique instrument identifier
        identifiers={"ClientInternal": models.InstrumentIdValue(instrument_id)},
        # our gilt instrument definition
        definition=fx_fwd_definition
    )}
    # Note we're using upsert_lusid_instrument and not upset_instrument as we're creating an instrument based
    # on a user defined instrument definition and not the base LUSID instruments.
    return instruments_api.upsert_lusid_instruments(bond_instrument_request)

gbpusd_fx_fwd_3y_instr_client_id = "gbp_usd_fwd_3y_12571"
instrument_creation_response = upsert_fx_fwd_instrument(gbpusd_fx_fwd_3y_instr_client_id, "GBP/USD 3Y @ 1.2571", gbpusd_fx_fwd_3y_instr_def)
# retrieve the instrument id of our gilt to be used later when loading market quotes for the bond into LUSID.
gbpusd_fx_fwd_3y_instr_luid = instrument_creation_response.values[gbpusd_fx_fwd_3y_instr_client_id].lusid_instrument_id

gbpusd_fx_fwd_1w_instr_client_id = "gbp_usd_fwd_1w_12509"
instrument_creation_response = upsert_fx_fwd_instrument(gbpusd_fx_fwd_1w_instr_client_id, "GBP/USD 1W @ 1.2509", gbpusd_fx_fwd_1w_instr_def)
# retrieve the instrument id of our gilt to be used later when loading market quotes for the bond into LUSID.
gbpusd_fx_fwd_1w_instr_luid = instrument_creation_response.values[gbpusd_fx_fwd_1w_instr_client_id].lusid_instrument_id


### Setting up our FX Forward Transaction

Let's assume we're taking a position in the GBP/USD 3Y Fx Forward we setup. As the domestic and foreign notionals are
included in the instrument definition we simply need to setup a transaction type that gives use a "stock" holding in the instrument. Now, as
there is no cash exchanged the default "Buy" transaction is unsuitable so we'll have to create our own. For details
on movements take a look at the notebook [Generating holdings with the movements engine in LUSID](https://github.com/finbourne/sample-notebooks/blob/master/examples/use-cases/ibor/Generating%20holdings%20with%20the%20movements%20engine%20in%20LUSID.ipynb)


#### Creating a Transaction Type

For more information on transaction types have a read of [Configuring transaction types](https://support.finbourne.com/configuring-transaction-types) and try the
 TODO find notebook

We need a transaction that simply buys a unit of the forward with no cash movement that we'll call "FxFwdInstBuy"

In [15]:
create_txn_type_response = create_transaction_type_configuration(
    api_factory,
    alias=models.TransactionConfigurationTypeAlias(
        type="FxFwdInstBuy",
        description="Buy a unit of a pre defined FXForwardInstrument",
        transaction_class="Basic",
        transaction_group="default",
        transaction_roles="LongLonger",
    ),
    movements=[
        models.TransactionConfigurationMovementDataRequest(
            movement_types="StockMovement",
            side="Side1",
            direction=1,
            properties=None,
            mappings=None,
        )
    ],
)


#### Upserting our transaction

With out transaction type configured we can now upsert a transation in the 3Y forward. An important point to note is when valuing
the Fx forward the calculated PV is a unitised value. Therefore our units in the transaction should match our notional:

In [16]:
def upsert_fwd_transaction(scope, txn_id, txn_type, txn_ccy, instrument_id, trade_date, settlement_date, portfolio, units, transaction_price, consideration):
    gilt_transaction_request = models.TransactionRequest(
        transaction_id=txn_id,
        type=txn_type,
        instrument_identifiers={"Instrument/default/ClientInternal": instrument_id},
        transaction_date=trade_date.isoformat(),
        settlement_date=settlement_date.isoformat(),
        units=units,
        transaction_price=models.TransactionPrice(price=transaction_price, type="Price"),
        total_consideration=models.CurrencyAndAmount(amount=consideration, currency=txn_ccy),
        exchange_rate=1,
        transaction_currency=txn_ccy
    )

    response = api_factory.build(lusid.api.TransactionPortfoliosApi).upsert_transactions(scope=scope,
                                                                                         code=portfolio,
                                                                                         transaction_request=[gilt_transaction_request])

# Insert position in 3Y

# our units are 100000000 as the PV in the forward pricing is calculated for a unit instrument.
units = 100000000
transaction_price = 1.2571  # spot + 3y fwd spread
consideration = 100000000
# upsert the buy leg of the forward
#upsert_fwd_transaction(scope, "fx_fwd_3y_b_001", "FxFwdInstBuy", "GBP", gbpusd_fx_fwd_3y_instr_client_id, trade_date, maturity_dates['3Y'], portfolio, transaction_price, consideration)
#upsert_fwd_transaction(scope, "fx_fwd_3y_b_001", "FxFwdInstBuy_CM", "GBP", gbpusd_fx_fwd_3y_instr_client_id, trade_date, maturity_dates['3Y'], portfolio, transaction_price, consideration)


# Insert position in 1W
units = 100000000
transaction_price = 1.2509  # spot + 3y fwd spread
consideration = 100000000
# upsert the buy leg of the forward
#upsert_fwd_transaction(scope, "fx_fwd_1w_b_001", "FxFwdInstBuy", "GBP", gbpusd_fx_fwd_1w_instr_client_id, trade_date, maturity_dates['1W'], portfolio, transaction_price, consideration)
#upsert_fwd_transaction(scope, "fx_fwd_1w_b_001", "FxFwdInstBuy", "GBP", gbpusd_fx_fwd_1w_instr_client_id, trade_date, maturity_dates['1W'], portfolio, transaction_price, consideration)
#upsert_fwd_transaction(scope, "fx_fwd_1w_b_001_CM", "FxFwdInstBuy_CM", "GBP", gbpusd_fx_fwd_1w_instr_client_id, trade_date, maturity_dates['1W'], portfolio, transaction_price, consideration)



### Value Portfolio

With our portfolio now populated with a transaction in the 3Y forward we can

In [17]:
def run_fx_forward_pricing_aggregation_on_portfolio(scope, portfolio, recipe, effective_at):
    aggregation_request = models.AggregationRequest(
        effective_at=effective_at.isoformat(),
        inline_recipe=recipe,
        metrics=[
            models.AggregateSpec(key='Holding/default/PV',
                                 op='Value'),
            # models.AggregateSpec(key='Analytic/default/ValuationDate',
            #                              op='Value'),
            # models.AggregateSpec(key='Analytic/default/DomCcy',
            #                      op='Value'),
            # models.AggregateSpec(key='Analytic/default/FgnCcy',
            #                      op='Value'),
            # models.AggregateSpec(key='Analytic/default/StartDate',
            #                      op='Value'),
            # models.AggregateSpec(key='Analytic/default/MaturityDate',
            #                      op='Value'),
            # models.AggregateSpec(key='Portfolio/default/Name',
            #                      op='Value'),
            # models.AggregateSpec(key='Holding/default/Cost',
            #                      op='Value'),
            # models.AggregateSpec(key='Holding/default/Units',
            #                      op='Value'),
            #             models.AggregateSpec("Holding/default/Error", "Value")
        ]
    )

    return api_factory.build(lusid.api.AggregationApi).get_aggregation(scope=scope, code=portfolio,
                                                             aggregation_request=aggregation_request)


result_t = run_fx_forward_pricing_aggregation_on_portfolio(scope, portfolio, fx_forward_pricing_recipe, trade_date)
result_t_plus_one = run_fx_forward_pricing_aggregation_on_portfolio(scope, portfolio, fx_forward_pricing_recipe, trade_date_plus_one)
result_t_plus_two = run_fx_forward_pricing_aggregation_on_portfolio(scope, portfolio, fx_forward_pricing_recipe, trade_date_plus_two)

aggregation_result_to_dataframe([
    ['Portfolio PV', trade_date, result_t.data[0]['Holding/default/PV']],
    ['Portfolio PV', trade_date_plus_one, result_t_plus_one.data[0]['Holding/default/PV']],
    ['Portfolio PV', trade_date_plus_two, result_t_plus_two.data[0]['Holding/default/PV']]
])

,Name,Effective At,Value
0,Portfolio PV,01 Jun 2020,-7994.751870
1,Portfolio PV,02 Jun 2020,-651743.391309
2,Portfolio PV,03 Jun 2020,627571.508930


In [18]:
result_t = run_fx_forward_pricing_aggregation('3Y', trade_date, gbpusd_fx_fwd_3y_instr_def, fx_forward_pricing_recipe)
result_t_plus_one = run_fx_forward_pricing_aggregation('3Y', trade_date_plus_one, gbpusd_fx_fwd_3y_instr_def, fx_forward_pricing_recipe)
result_t_plus_two = run_fx_forward_pricing_aggregation('3Y', trade_date_plus_two, gbpusd_fx_fwd_3y_instr_def, fx_forward_pricing_recipe)

aggregation_result_to_dataframe([
    ['GBP/USD 3Y Forward PV', trade_date, result_t.data[0]['Holding/default/PV']],
    ['GBP/USD 3Y Forward PV', trade_date_plus_one, result_t_plus_one.data[0]['Holding/default/PV']],
    ['GBP/USD 3Y Forward PV', trade_date_plus_two, result_t_plus_two.data[0]['Holding/default/PV']]
])

,Name,Effective At,Value
0,GBP/USD 3Y Forward PV,01 Jun 2020,-0.005022
1,GBP/USD 3Y Forward PV,02 Jun 2020,-0.011472
2,GBP/USD 3Y Forward PV,03 Jun 2020,0.001346


### Price across the life of the forward

In [19]:
# assume spot rate on expiry date is lower thant forward rate
upsert_gbp_usd_fx_rate(1.2509, maturity_dates['1W'])


result_t = run_fx_forward_pricing_aggregation('1W', trade_date, gbpusd_fx_fwd_1w_instr_def, fx_forward_pricing_recipe)
result_t_plus_one = run_fx_forward_pricing_aggregation('1W', trade_date_plus_one, gbpusd_fx_fwd_1w_instr_def, fx_forward_pricing_recipe)
result_on_maturity = run_fx_forward_pricing_aggregation('1W', maturity_dates['1W'], gbpusd_fx_fwd_1w_instr_def, fx_forward_pricing_recipe)

aggregation_result_to_dataframe([
    ['GBP/USD 1W Forward PV', trade_date, result_t.data[0]['Holding/default/PV']],
    ['GBP/USD 1W Forward PV', trade_date_plus_one, result_t_plus_one.data[0]['Holding/default/PV']],
    ['GBP/USD 1W Forward PV', maturity_dates['1W'], result_on_maturity.data[0]['Holding/default/PV']]
])


'GBP/USD @ 1.2509 for 2020-06-08 00:00:00+00:00 uploaded to quote store.'

'USD/GBP @ 0.7994244144216165 for 2020-06-08 00:00:00+00:00 uploaded to quote store.'

,Name,Effective At,Value
0,GBP/USD 1W Forward PV,01 Jun 2020,-0.000080
1,GBP/USD 1W Forward PV,02 Jun 2020,-0.006517
2,GBP/USD 1W Forward PV,08 Jun 2020,0.000000


## Examples To Tests

Example Set  1 :
* Transactions modelled as "CashForward" movements that only ever impact Side 1. So it's a buy and a sell
* For test we assume Spot Price at expiry above the Forward rate to ensure a Pnl exists
* Tests :
  * Example 1a : One transaction only. We buy contract (so buy domestic and sell foreign) (FxFwdInstBuy_CM)
  * Example 1b : One transaction only. We sell contract (sell domestic, buy foreign)  (FxFwdInstSell_CM)
  * Example 1c : Two transactions. We buy contract (so buy domestic and sell foreign) (FxFwdInstBuy_CM followed FxFwdInstSell_CM)
  * Example 1d : Two transactions. We sell contract (so sell domestic, buy foreign) (FxFwdInstSell_CM followed FxFwdInstBuy_CM)

Example Set 2 :
*Transactions modelled using defaults



## Transactions to be used


In [20]:
create_txn_type_response = create_transaction_type_configuration(
    api_factory,
    alias=models.TransactionConfigurationTypeAlias(
        type="FxFwdInstBuy_CM",
        description="Buy a unit of a pre defined FXForwardInstrument",
        transaction_class="Basic",
        transaction_group="default",
        transaction_roles="Longer",
    ),
    movements=[
        models.TransactionConfigurationMovementDataRequest(
            movement_types="CashFxForward",
            side="Side1",
            direction=1,
            properties=None,
            mappings=None,
        )
    ],
)

create_txn_type_response = create_transaction_type_configuration(
    api_factory,
    alias=models.TransactionConfigurationTypeAlias(
        type="FxFwdInstSell_CM",
        description="Sell a unit of a pre defined FXForwardInstrument",
        transaction_class="Basic",
        transaction_group="default",
        transaction_roles="Longer",
    ),
    movements=[
        models.TransactionConfigurationMovementDataRequest(
            movement_types="CashFxForward",
            side="Side1",
            direction=-1,
            properties=None,
            mappings=None,
        )
    ],
)

## Examples :

Recall parameters for GBP/USD 1Wk Fwd
Trade Date : 1 June 2020 (Spot Rate : 1.2508)
Settlement Date: 8 June 2020 (Spot Rate : set in examples)
Dom : 100000000
Fgn : 125090000
Fwd Rate : 1.2509

### Example 1 :
* We only upsert one transaction which is a buy of the 1W GBP/USD forward.
* FxFwdInstBuy_CM transaction type is a "CashFxForward" Side 1 Movement only.
* On settlement Forward becomes a "P" holiding with no corresponding cash holding in underlying currencies

In [21]:
portfolio_cfm_buy_only = "fxfwd-portfolio-cfm-buy-only"
create_portfolio(scope, portfolio_cfm_buy_only, portfolio_cfm_buy_only, "GBP")

# spot on fx forward 1.2508 on trade date
upsert_gbp_usd_fx_rate(1.2508, trade_date)

units = 10000
transaction_price = 1.2509  # spot + 3y fwd spread
consideration = 10000

# buy transaction
upsert_fwd_transaction(scope, "fx_fwd_1w_cfm_buy_only_001", "FxFwdInstBuy_CM", "GBP", gbpusd_fx_fwd_1w_instr_client_id, trade_date, maturity_dates['1W'], portfolio_cfm_buy_only, units, transaction_price, consideration)

# assume spot rate on expiry date is higher than the forward rate
upsert_gbp_usd_fx_rate(1.2609, maturity_dates['1W'])

# run aggregations
result_t = run_fx_forward_pricing_aggregation_on_portfolio(scope, portfolio_cfm_buy_only, fx_forward_pricing_recipe, trade_date)
result_expiry = run_fx_forward_pricing_aggregation_on_portfolio(scope, portfolio_cfm_buy_only, fx_forward_pricing_recipe, maturity_dates['1W'])

aggregation_result_to_dataframe([
    ['Portfolio PV (CFM Buy Only)', trade_date, result_t.data[0]['Holding/default/PV']],
    ['Portfolio PV (CFM Buy Only)', maturity_dates['1W'], result_expiry.data[0]['Holding/default/PV']]
])

'GBP/USD @ 1.2508 for 2020-06-01 00:00:00+00:00 uploaded to quote store.'

'USD/GBP @ 0.799488327470419 for 2020-06-01 00:00:00+00:00 uploaded to quote store.'

'GBP/USD @ 1.2609 for 2020-06-08 00:00:00+00:00 uploaded to quote store.'

'USD/GBP @ 0.7930843048616069 for 2020-06-08 00:00:00+00:00 uploaded to quote store.'

,Name,Effective At,Value
0,Portfolio PV (CFM Buy Only),01 Jun 2020,-0.799475
1,Portfolio PV (CFM Buy Only),08 Jun 2020,79.307127


#### Holdings

In [22]:
holdings_response_trade_date = transaction_portfolios_api.get_holdings(
    scope=scope, code=portfolio_cfm_buy_only, property_keys=["Instrument/default/Name"], effective_at=trade_date
)

trade_date_holdings = display_holdings_summary(holdings_response_trade_date, trade_date)

holdings_response_expiry = transaction_portfolios_api.get_holdings(
    scope=scope, code=portfolio_cfm_buy_only, property_keys=["Instrument/default/Name"], effective_at=maturity_dates['1W']
)

expiry_date_holdings = display_holdings_summary(holdings_response_expiry, maturity_dates['1W'])

pd.concat([trade_date_holdings, expiry_date_holdings])

,names,effectiveAt,units,settled_units,holding types,cost
0,GBP/USD 1W @ 1.2509,2020-06-01 00:00:00+00:00,10000.000000,0.000000,F,"{'amount': 10000.0, 'currency': 'GBP'}"
0,GBP/USD 1W @ 1.2509,2020-06-08 00:00:00+00:00,10000.000000,10000.000000,P,"{'amount': 10000.0, 'currency': 'GBP'}"


Forward type before expiry becomes a position  -> but no auto matching cash transactions!



### Example 2:
* We upsert two transactions:
  * Buy of the 1W GBP/USD forward in domestic ccy (so 10,000GBP notional)
  * Sell of the 1W GBP/USD forward in foreign ccy (so 12,509USD notional)
* FxFwdInstBuy_CM transaction type is a "CashFxForward" Side 1 Dir 1 Movement only.
* FxFwdInstSell_CM transaction type is a "CashFxForward" Side 1 Dir -1 Movement only.
* On settlement data Forward become holding positions

In [23]:
portfolio_cfm_buy_sell_trade_date = "fxfwd-portfolio-cfm-buy-sell-trade-date"
create_portfolio(scope, portfolio_cfm_buy_sell_trade_date, portfolio_cfm_buy_sell_trade_date, "GBP")

# spot on fx forward 1.2508 on trade date
upsert_gbp_usd_fx_rate(1.2508, trade_date)

# domestic (Buy)
units = 10000
transaction_price = 1.2509  # spot + 3y fwd spread
consideration = 10000

# buy transaction
upsert_fwd_transaction(scope, "fx_fwd_1w_cfm_buy_trade_date_001", "FxFwdInstBuy_CM", "GBP", gbpusd_fx_fwd_1w_instr_client_id, trade_date, maturity_dates['1W'], portfolio_cfm_buy_sell_trade_date, units, transaction_price, consideration)

# foreign (Sell)
units = 12509
transaction_price = 1.2509  # spot + 3y fwd spread
consideration = 12509

# sell transaction
upsert_fwd_transaction(scope, "fx_fwd_1w_cfm_sell_trade_date_001", "FxFwdInstSell_CM", "USD", gbpusd_fx_fwd_1w_instr_client_id, trade_date, maturity_dates['1W'], portfolio_cfm_buy_sell_trade_date, units, transaction_price, consideration)

# assume spot rate on expiry date is higher thant forward rate
upsert_gbp_usd_fx_rate(1.2609, maturity_dates['1W'])

# run aggregations
result_t = run_fx_forward_pricing_aggregation_on_portfolio(scope, portfolio_cfm_buy_sell_trade_date, fx_forward_pricing_recipe, trade_date)
result_expiry = run_fx_forward_pricing_aggregation_on_portfolio(scope, portfolio_cfm_buy_sell_trade_date, fx_forward_pricing_recipe, maturity_dates['1W'])

aggregation_result_to_dataframe([
    ['Portfolio PV (CFM Buy and Sell On Trade Date)', trade_date, result_t.data[0]['Holding/default/PV']],
    ['Portfolio PV (CFM Buy and Sell On Trade Date)', maturity_dates['1W'], result_expiry.data[0]['Holding/default/PV']]
])

'GBP/USD @ 1.2508 for 2020-06-01 00:00:00+00:00 uploaded to quote store.'

'USD/GBP @ 0.799488327470419 for 2020-06-01 00:00:00+00:00 uploaded to quote store.'

'GBP/USD @ 1.2609 for 2020-06-08 00:00:00+00:00 uploaded to quote store.'

'USD/GBP @ 0.7930843048616069 for 2020-06-08 00:00:00+00:00 uploaded to quote store.'

,Name,Effective At,Value
0,Portfolio PV (CFM Buy and Sell On Trade Date),01 Jun 2020,-0.799475
1,Portfolio PV (CFM Buy and Sell On Trade Date),08 Jun 2020,79.307127


#### Example Holdings

In [24]:
holdings_response_trade_date = transaction_portfolios_api.get_holdings(
    scope=scope, code=portfolio_cfm_buy_sell_trade_date, property_keys=["Instrument/default/Name"], effective_at=trade_date
)

trade_date_holdings = display_holdings_summary(holdings_response_trade_date, trade_date)

holdings_response_expiry = transaction_portfolios_api.get_holdings(
    scope=scope, code=portfolio_cfm_buy_sell_trade_date, property_keys=["Instrument/default/Name"], effective_at=maturity_dates['1W']
)

expiry_date_holdings = display_holdings_summary(holdings_response_expiry, maturity_dates['1W'])

pd.concat([trade_date_holdings, expiry_date_holdings])



,names,effectiveAt,units,settled_units,holding types,cost
0,GBP/USD 1W @ 1.2509,2020-06-01 00:00:00+00:00,10000.000000,0.000000,F,"{'amount': 10000.0, 'currency': 'GBP'}"
1,GBP/USD 1W @ 1.2509,2020-06-01 00:00:00+00:00,-12509.000000,0.000000,F,"{'amount': -12509.0, 'currency': 'USD'}"
0,GBP/USD 1W @ 1.2509,2020-06-08 00:00:00+00:00,10000.000000,10000.000000,P,"{'amount': 10000.0, 'currency': 'GBP'}"
1,GBP/USD 1W @ 1.2509,2020-06-08 00:00:00+00:00,-12509.000000,-12509.000000,P,"{'amount': -12509.0, 'currency': 'USD'}"


### Example 3

* We only upsert one transaction which is a buy of the 1W GBP/USD forward.
* FxFwdBuy transaction type is a "CashFxForward" Side 1, Dir 1 Movement and a "CashFxForward" Side 2, Dir -1
* On settlement Forward becomes a "P" holding with a corresponding cash holding in underlying in the transaction currency (-10000 CCY_GBP)

In [25]:
portfolio_default_buy_only = "fxfwd-portfolio-default-buy-only"
create_portfolio(scope, portfolio_default_buy_only, portfolio_default_buy_only, "GBP")

# spot on fx forward 1.2508 on trade date
upsert_gbp_usd_fx_rate(1.2508, trade_date)

# domestic (Buy)
units = 10000
transaction_price = 1.2509  # spot + 3y fwd spread) // TODO ask on rounding in web?
consideration = 10000

# buy transaction
upsert_fwd_transaction(scope, "fx_fwd_1w_cfm_buy_trade_date_001", "FwdFxBuy", "GBP", gbpusd_fx_fwd_1w_instr_client_id, trade_date, maturity_dates['1W'], portfolio_default_buy_only, units, transaction_price, consideration)

# assume spot rate on expiry date is higher thant forward rate
upsert_gbp_usd_fx_rate(1.2609, maturity_dates['1W'])

# run aggregations
result_t = run_fx_forward_pricing_aggregation_on_portfolio(scope, portfolio_default_buy_only, fx_forward_pricing_recipe, trade_date)
result_expiry = run_fx_forward_pricing_aggregation_on_portfolio(scope, portfolio_default_buy_only, fx_forward_pricing_recipe, maturity_dates['1W'])

aggregation_result_to_dataframe([
    ['Portfolio PV (CFM Buy and Sell On Trade Date)', trade_date, result_t.data[0]['Holding/default/PV']],
    ['Portfolio PV (CFM Buy and Sell On Trade Date)', maturity_dates['1W'], result_expiry.data[0]['Holding/default/PV']]
])

'GBP/USD @ 1.2508 for 2020-06-01 00:00:00+00:00 uploaded to quote store.'

'USD/GBP @ 0.799488327470419 for 2020-06-01 00:00:00+00:00 uploaded to quote store.'

'GBP/USD @ 1.2609 for 2020-06-08 00:00:00+00:00 uploaded to quote store.'

'USD/GBP @ 0.7930843048616069 for 2020-06-08 00:00:00+00:00 uploaded to quote store.'

,Name,Effective At,Value
0,Portfolio PV (CFM Buy and Sell On Trade Date),01 Jun 2020,-0.799475
1,Portfolio PV (CFM Buy and Sell On Trade Date),08 Jun 2020,79.307127


#### Holdings

In [26]:
holdings_response_trade_date = transaction_portfolios_api.get_holdings(
    scope=scope, code=portfolio_default_buy_only, property_keys=["Instrument/default/Name"], effective_at=trade_date
)

trade_date_holdings = display_holdings_summary(holdings_response_trade_date, trade_date)

holdings_response_expiry = transaction_portfolios_api.get_holdings(
    scope=scope, code=portfolio_default_buy_only, property_keys=["Instrument/default/Name"], effective_at=maturity_dates['1W']
)

expiry_date_holdings = display_holdings_summary(holdings_response_expiry, maturity_dates['1W'])

pd.concat([trade_date_holdings, expiry_date_holdings])





,names,effectiveAt,units,settled_units,holding types,cost
0,GBP/USD 1W @ 1.2509,2020-06-01 00:00:00+00:00,10000.000000,0.000000,F,"{'amount': 10000.0, 'currency': 'GBP'}"
1,CCY_GBP,2020-06-01 00:00:00+00:00,-10000.000000,0.000000,F,"{'amount': -10000.0, 'currency': 'GBP'}"
0,GBP/USD 1W @ 1.2509,2020-06-08 00:00:00+00:00,10000.000000,10000.000000,P,"{'amount': 10000.0, 'currency': 'GBP'}"
1,CCY_GBP,2020-06-08 00:00:00+00:00,-10000.000000,-10000.000000,B,"{'amount': -10000.0, 'currency': 'GBP'}"


### Example 4
* We upsert two transactions:
  * Buy of the 1W GBP/USD forward in domestic ccy (so 10,000GBP notional)
  * Sell of the 1W GBP/USD forward in foreign ccy (so 12,509USD notional)
* FxFwdBuy transaction type is a "CashFxForward" Side 1 Dir 1 Movement only.
* FxFwdSell transaction type is a "CashFxForward" Side 1 Dir -1 Movement only.
* On settlement data Forward become holding positions

In [27]:
portfolio_default_buy_sell_only = "fxfwd-portfolio-default-buy-sell-only"
create_portfolio(scope, portfolio_default_buy_sell_only, portfolio_default_buy_sell_only, "GBP")

# spot on fx forward 1.2508 on trade date
upsert_gbp_usd_fx_rate(1.2508, trade_date)

# domestic (Buy)
units = 10000
transaction_price = 1.2509  # spot + 3y fwd spread
consideration = 10000

# buy transaction
upsert_fwd_transaction(scope, "fx_fwd_1w_cfm_buy_trade_date_001", "FwdFxBuy", "GBP", gbpusd_fx_fwd_1w_instr_client_id, trade_date, maturity_dates['1W'], portfolio_default_buy_sell_only, units, transaction_price, consideration)

# foreign (Sell)
units = 12509
transaction_price = 1.2509  # spot + 3y fwd spread
consideration = 12509

# sell transaction
upsert_fwd_transaction(scope, "fx_fwd_1w_cfm_sell_trade_date_001", "FwdFxSell", "USD", gbpusd_fx_fwd_1w_instr_client_id, trade_date, maturity_dates['1W'], portfolio_default_buy_sell_only, units, transaction_price, consideration)

# assume spot rate on expiry date is higher thant forward rate
upsert_gbp_usd_fx_rate(1.2609, maturity_dates['1W'])

# run aggregations
result_t = run_fx_forward_pricing_aggregation_on_portfolio(scope, portfolio_default_buy_sell_only, fx_forward_pricing_recipe, trade_date)
result_expiry = run_fx_forward_pricing_aggregation_on_portfolio(scope, portfolio_default_buy_sell_only, fx_forward_pricing_recipe, maturity_dates['1W'])


aggregation_result_to_dataframe([
    ['Portfolio PV (CFM Buy and Sell On Trade Date)', trade_date, result_t.data[0]['Holding/default/PV']],
    ['Portfolio PV (CFM Buy and Sell On Trade Date)', maturity_dates['1W'], result_expiry.data[0]['Holding/default/PV']]
])

'GBP/USD @ 1.2508 for 2020-06-01 00:00:00+00:00 uploaded to quote store.'

'USD/GBP @ 0.799488327470419 for 2020-06-01 00:00:00+00:00 uploaded to quote store.'

'GBP/USD @ 1.2609 for 2020-06-08 00:00:00+00:00 uploaded to quote store.'

'USD/GBP @ 0.7930843048616069 for 2020-06-08 00:00:00+00:00 uploaded to quote store.'

,Name,Effective At,Value
0,Portfolio PV (CFM Buy and Sell On Trade Date),01 Jun 2020,-0.799475
1,Portfolio PV (CFM Buy and Sell On Trade Date),08 Jun 2020,79.307127


#### Holdings

In [28]:
holdings_response_trade_date = transaction_portfolios_api.get_holdings(
    scope=scope, code=portfolio_default_buy_sell_only, property_keys=["Instrument/default/Name"], effective_at=trade_date
)

holdings_response_expiry = transaction_portfolios_api.get_holdings(
    scope=scope, code=portfolio_default_buy_sell_only, property_keys=["Instrument/default/Name"], effective_at=maturity_dates['1W']
)

expiry_date_holdings = display_holdings_summary(holdings_response_expiry, maturity_dates['1W'])

pd.concat([trade_date_holdings, expiry_date_holdings])

,names,effectiveAt,units,settled_units,holding types,cost
0,GBP/USD 1W @ 1.2509,2020-06-01 00:00:00+00:00,10000.000000,0.000000,F,"{'amount': 10000.0, 'currency': 'GBP'}"
1,CCY_GBP,2020-06-01 00:00:00+00:00,-10000.000000,0.000000,F,"{'amount': -10000.0, 'currency': 'GBP'}"
0,GBP/USD 1W @ 1.2509,2020-06-08 00:00:00+00:00,10000.000000,10000.000000,P,"{'amount': 10000.0, 'currency': 'GBP'}"
1,GBP/USD 1W @ 1.2509,2020-06-08 00:00:00+00:00,-12509.000000,-12509.000000,P,"{'amount': -12509.0, 'currency': 'USD'}"
2,CCY_GBP,2020-06-08 00:00:00+00:00,-10000.000000,-10000.000000,B,"{'amount': -10000.0, 'currency': 'GBP'}"
3,CCY_USD,2020-06-08 00:00:00+00:00,12509.000000,12509.000000,B,"{'amount': 12509.0, 'currency': 'USD'}"


Fwd becomes a posittion as well as cash balance. Why?? We have our cash positions the contract is done




### Example 3 : Stock Movement only a buy transaction

In [29]:
portfolio_sm_buy_only = "fxfwd-portfolio-sm-buy-only"
create_portfolio(scope, portfolio_sm_buy_only, portfolio_sm_buy_only, "GBP")

units = 10000
transaction_price = 1.2509  # spot + 3y fwd spread
consideration = 10000

# buy transaction
upsert_fwd_transaction(scope, "fx_fwd_1w_cfm_sm_only_001", "FxFwdInstBuy", "GBP", gbpusd_fx_fwd_1w_instr_client_id, trade_date, maturity_dates['1W'], portfolio_sm_buy_only, units, transaction_price, consideration)

# assume spot rate on expiry date is higher thant forward rate
upsert_gbp_usd_fx_rate(1.2609, maturity_dates['1W'])

# run aggregations
result_t = run_fx_forward_pricing_aggregation_on_portfolio(scope, portfolio_sm_buy_only, fx_forward_pricing_recipe, trade_date)
result_expiry = run_fx_forward_pricing_aggregation_on_portfolio(scope, portfolio_sm_buy_only, fx_forward_pricing_recipe, maturity_dates['1W'])

aggregation_result_to_dataframe([
    ['Portfolio PV (CFM Buy Only)', trade_date, result_t.data[0]['Holding/default/PV']],
    ['Portfolio PV (CFM Buy Only)', maturity_dates['1W'], result_expiry.data[0]['Holding/default/PV']]
])

'GBP/USD @ 1.2609 for 2020-06-08 00:00:00+00:00 uploaded to quote store.'

'USD/GBP @ 0.7930843048616069 for 2020-06-08 00:00:00+00:00 uploaded to quote store.'

,Name,Effective At,Value
0,Portfolio PV (CFM Buy Only),01 Jun 2020,-0.799475
1,Portfolio PV (CFM Buy Only),08 Jun 2020,79.307127


#### Example 3a Holdings

In [30]:
holdings_response_trade_date = transaction_portfolios_api.get_holdings(
    scope=scope, code=portfolio_sm_buy_only, property_keys=["Instrument/default/Name"], effective_at=trade_date
)

trade_date_holdings = display_holdings_summary(holdings_response_trade_date, trade_date)

holdings_response_expiry = transaction_portfolios_api.get_holdings(
    scope=scope, code=portfolio_sm_buy_only, property_keys=["Instrument/default/Name"], effective_at=maturity_dates['1W']
)

expiry_date_holdings = display_holdings_summary(holdings_response_expiry, maturity_dates['1W'])

pd.concat([trade_date_holdings, expiry_date_holdings])




,names,effectiveAt,units,settled_units,holding types,cost
0,GBP/USD 1W @ 1.2509,2020-06-01 00:00:00+00:00,10000.000000,0.000000,P,"{'amount': 10000.0, 'currency': 'GBP'}"
0,GBP/USD 1W @ 1.2509,2020-06-08 00:00:00+00:00,10000.000000,10000.000000,P,"{'amount': 10000.0, 'currency': 'GBP'}"


### Example 4 :
* We upsert two transactions:
  * Trade Date : Buy of the GBP/USD 1W forward in domestic ccy (so 10,000GBP notional)
  * Maturity Date : Close position and delivery the cash
* FxFwdBuy_Otc transaction type:
  * StockMovement Side 1 Dir 1 Movement only (Positoin unsettled units)
* FxFwdSell_Otc transaction type:  (assumes cash actually delivered on settlement baked into one transaction)
  * StockMovement Side 1 Dir -1 to close out Position
  * Cash Side 2 Dir -1 outgoing domestic
  * Cash Side 2 Dir 1  incoming foreign
* On settlement data Forward become holding positions



#### Define Transaction Types

In [45]:
def upsert_cash_transaction(scope, portfolio, units, transaction_price, consideration, txn_id, txn_type, txn_ccy, transaction_date, exchange_rate=1):
    gilt_transaction_request = models.TransactionRequest(
        transaction_id=txn_id,
        type=txn_type,
        instrument_identifiers={"Instrument/default/Currency": txn_ccy},
        transaction_date=transaction_date.isoformat(),
        settlement_date=transaction_date.isoformat(),
        units=units,
        transaction_price=models.TransactionPrice(price=transaction_price, type="Price"),
        total_consideration=models.CurrencyAndAmount(amount=consideration, currency=txn_ccy),
        exchange_rate=exchange_rate,
        transaction_currency=txn_ccy,
    )

    response = api_factory.build(lusid.api.TransactionPortfoliosApi).upsert_transactions(scope=scope,
                                                                                         code=portfolio,
                                                                                         transaction_request=[gilt_transaction_request])


def upsert_fwd_close_transaction(scope, portfolio, units, transaction_price, consideration, txn_id, txn_type, txn_ccy, transaction_date, exchange_rate=1):
    gilt_transaction_request = models.TransactionRequest(
        transaction_id=txn_id,
        type=txn_type,
        instrument_identifiers={"Instrument/default/Currency": txn_ccy},
        transaction_date=transaction_date.isoformat(),
        settlement_date=transaction_date.isoformat(),
        units=units,
        transaction_price=models.TransactionPrice(price=transaction_price, type="Price"),
        total_consideration=models.CurrencyAndAmount(amount=consideration, currency=txn_ccy),
        exchange_rate=exchange_rate,
        transaction_currency=txn_ccy,
    )

    response = api_factory.build(lusid.api.TransactionPortfoliosApi).upsert_transactions(scope=scope,
                                                                                         code=portfolio,
                                                                                         transaction_request=[gilt_transaction_request])


In [ ]:
create_txn_type_response = create_transaction_type_configuration(
    api_factory,
    alias=models.TransactionConfigurationTypeAlias(
        type="FxFwdOpen_Otc",
        description="Open a position in an  FX Forward Instrument",
        transaction_class="Basic",
        transaction_group="default",
        transaction_roles="Longer",
    ),
    movements=[
        models.TransactionConfigurationMovementDataRequest(
            movement_types="StockMovement",
            side="Side1",
            direction=1
        )
    ],
)

create_txn_type_response = create_transaction_type_configuration(
    api_factory,
    alias=models.TransactionConfigurationTypeAlias(
        type="FxFwdClose_Otc",
        description="Close a position in an  FX Forward Instrument",
        transaction_class="Basic",
        transaction_group="default",
        transaction_roles="Longer",
    ),
    movements=[
        models.TransactionConfigurationMovementDataRequest(
            movement_types="StockMovement",
            side="Side1",
            direction=-1
        )
    ],
)

create_txn_type_response = create_transaction_type_configuration(
    api_factory,
    alias=models.TransactionConfigurationTypeAlias(
        type="FxFwdClose_Otc_DomCash",
        description="Close a position in an  FX Forward Instrument - Domestic cash delivery",
        transaction_class="Basic",
        transaction_group="default",
        transaction_roles="Shorter",
    ),
    movements=[
        models.TransactionConfigurationMovementDataRequest(
            movement_types="CashSettlement",
            side="Side1",
            direction=-1
        )
    ],
)

create_txn_type_response = create_transaction_type_configuration(
    api_factory,
    alias=models.TransactionConfigurationTypeAlias(
        type="FxFwdClose_Otc_FgnCash",
        description="Close a position in an  FX Forward Instrument - Foreign cash receivable",
        transaction_class="Basic",
        transaction_group="default",
        transaction_roles="Longer",
    ),
    movements=[
        models.TransactionConfigurationMovementDataRequest(
            movement_types="CashSettlement",
            side="Side1",
            direction=1
        )
    ],
)

In [69]:
def create_fx_fwd_fgn_cash_side():
    fgn_cash_side_cfg = models.SideConfigurationDataRequest(
        side="FxFwdFgnCashSide_02",
        security=f"Transaction/{scope}/FgnCcy",
        currency="Txn:TradeCurrency",
        rate="Txn:TradeToPortfolioRate",
        units="Txn:TradeAmount",
        amount="Txn:TradeAmount"
    )

    response = system_configuration_api.create_side_definition(side_configuration_data_request=fgn_cash_side_cfg)
    return response
create_fx_fwd_fgn_cash_side()

{'links': [{'description': None,
            'href': 'https://khalid-local-dev.lusid.com/api/api/schemas/entities/SideConfigurationData',
            'method': 'GET',
            'relation': 'EntitySchema'},
           {'description': 'A link to the LUSID Insights website showing all '
                           'logs related to this request',
            'href': 'http://khalid-local-dev.lusid.com/app/insights/logs/0HM1381C5JQ3P:00000001',
            'method': 'GET',
            'relation': 'RequestLogs'}],
 'side_definitions': [{'amount': 'Txn:TradeAmount',
                       'currency': 'Txn:TradeCurrency',
                       'links': None,
                       'rate': 'Txn:TradeToPortfolioRate',
                       'security': 'Txn:LusidInstrumentId',
                       'side': 'Side1',
                       'units': 'Txn:Units'},
                      {'amount': 'Txn:TotalConsideration',
                       'currency': 'Txn:SettlementCurrency',
               

In [66]:
def create_instrument_mapping_prop():
    return  api_factory.build(lusid.api.PropertyDefinitionsApi).create_property_definition(
        create_property_definition_request=lusid.models.CreatePropertyDefinitionRequest(
            domain = "Transaction",
            scope = scope,
            code = "FgnCcy",
            value_required = None,
            display_name = "FX Forward CCY",
            data_type_id = lusid.ResourceId(scope="system", code="string"),
            life_time = None
        )
    )

create_instrument_mapping_prop()

{'code': 'FgnCcy',
 'constraint_style': 'Property',
 'data_type_id': {'code': 'string', 'scope': 'system'},
 'display_name': 'FX Forward CCY',
 'domain': 'Transaction',
 'href': None,
 'key': 'Transaction/fx-forward-pricing-nb/FgnCcy',
 'life_time': 'Perpetual',
 'links': [{'description': None,
            'href': 'https://khalid-local-dev.lusid.com/api/api/schemas/entities/PropertyDefinition',
            'method': 'GET',
            'relation': 'EntitySchema'},
           {'description': 'A link to the LUSID Insights website showing all '
                           'logs related to this request',
            'href': 'http://khalid-local-dev.lusid.com/app/insights/logs/0HM1381J6LKBL:00000001',
            'method': 'GET',
            'relation': 'RequestLogs'}],
 'scope': 'fx-forward-pricing-nb',
 'type': 'Label',
 'unit_schema': 'NoUnits',
 'value_required': False,
 'value_type': 'String'}

In [70]:
create_transaction_type_configuration(
    api_factory,
    alias=models.TransactionConfigurationTypeAlias(
        type="FxFwdClose_Otc_All_2",
        description="Close a position in an  FX Forward Instrument",
        transaction_class="Basic",
        transaction_group="default",
        transaction_roles="Longer",
    ),
    movements=[
        models.TransactionConfigurationMovementDataRequest(
            movement_types="StockMovement",
            side="Side1",
            direction=-1
        ),
        models.TransactionConfigurationMovementDataRequest(
            movement_types="CashSettlement",
            side="Side2",
            direction=-1
        ),
        models.TransactionConfigurationMovementDataRequest(
            movement_types="CashSettlement",
            side="FxFwdFgnCashSide_02",
            direction=1
        )
    ],
)


{'links': [{'description': None,
            'href': 'https://khalid-local-dev.lusid.com/api/api/schemas/entities/TransactionConfigurationData',
            'method': 'GET',
            'relation': 'EntitySchema'},
           {'description': None,
            'href': 'https://khalid-local-dev.lusid.com/api/',
            'method': 'GET',
            'relation': 'PropertySchema'},
           {'description': 'A link to the LUSID Insights website showing all '
                           'logs related to this request',
            'href': 'http://khalid-local-dev.lusid.com/app/insights/logs/0HM1381J6LKE5:00000002',
            'method': 'GET',
            'relation': 'RequestLogs'}],
 'side_definitions': [{'amount': 'Txn:TradeAmount',
                       'currency': 'Txn:TradeCurrency',
                       'links': None,
                       'rate': 'Txn:TradeToPortfolioRate',
                       'security': 'Txn:LusidInstrumentId',
                       'side': 'Side1',
      

In [84]:
portfolio_otc = "fxfwd-portfolio_otc_02"
create_portfolio(scope, portfolio_otc, portfolio_otc, "GBP")

fwd_units = 10000
transaction_price = 1.2509  # spot + 3y fwd spread
consideration = 10000
fgn_consideration = 10000 * 1.2509 # should I retrieve this quote from quote store? or insturment def? or this ok to say just replicate a cash transaction from extern system
dom_units_on_settlement = 10000
fgn_units_on_settlement = 12509

# assume spot rate on expiry date is higher thant forward rate
upsert_gbp_usd_fx_rate(1.2609, maturity_dates['1W'])

# buy transaction on June 1
upsert_fwd_transaction(scope, "fx_fwd_1w_open_001", "FxFwdOpen_Otc", "GBP", gbpusd_fx_fwd_1w_instr_client_id, trade_date, trade_date, portfolio_otc, units, transaction_price, consideration)





'GBP/USD @ 1.2609 for 2020-06-08 00:00:00+00:00 uploaded to quote store.'

'USD/GBP @ 0.7930843048616069 for 2020-06-08 00:00:00+00:00 uploaded to quote store.'

#### Open Forward Positions

In [85]:
t_p_1 = datetime(2020, 6, 2, tzinfo=pytz.utc),
t_p_3 = datetime(2020, 6, 4, tzinfo=pytz.utc),


# run aggregations
result_t = run_fx_forward_pricing_aggregation_on_portfolio(scope, portfolio_otc, fx_forward_pricing_recipe, trade_date)
result_t_p_1 = run_fx_forward_pricing_aggregation_on_portfolio(scope, portfolio_otc, fx_forward_pricing_recipe, t_p_1)
result_t_p_3 = run_fx_forward_pricing_aggregation_on_portfolio(scope, portfolio_otc, fx_forward_pricing_recipe, t_p_3)
result_expiry = run_fx_forward_pricing_aggregation_on_portfolio(scope, portfolio_otc, fx_forward_pricing_recipe, maturity_dates['1W'])

aggregation_result_to_dataframe([
    ['Portfolio PV (CFM Buy Only)', trade_date, result_t.data[0]['Holding/default/PV']],
    ['Portfolio PV (CFM Buy Only)', maturity_dates['1W'], result_expiry.data[0]['Holding/default/PV']]
])

AttributeError: 'tuple' object has no attribute 'isoformat'

#### On Settlement

In [91]:
# close forward on Jun 8
#upsert_fwd_transaction(scope, "fx_fwd_1w_close_001", "FxFwdClose_Otc", "GBP", gbpusd_fx_fwd_1w_instr_client_id, maturity_dates['1W'], maturity_dates['1W'], portfolio_otc, units, transaction_price, consideration)
# exchange cash flows on Jun 8
#upsert_cash_transaction(scope, portfolio_otc, dom_units_on_settlement, transaction_price, consideration, "fx_fwd_1w_close_001_pay_dom", "FxFwdClose_Otc_DomCash", "GBP", maturity_dates['1W'])
# has to be in gbp with exch rate. otherwise cost calculated from units
#upsert_cash_transaction(scope, portfolio_otc, units, transaction_price, fgn_consideration, "fx_fwd_1w_close_001_rec_fgn", "FxFwdClose_Otc_FgnCash", "USD", maturity_dates['1W'])
#upsert_cash_transaction(scope, portfolio_otc, fgn_units_on_settlement, transaction_price, fgn_consideration, "fx_fwd_1w_close_001_rec_fgn", "FxFwdClose_Otc_FgnCash", "USD", maturity_dates['1W'])

usd_gbp_contract_rate = 1/1.2509
#upsert_fwd_transaction(scope, "fx_fwd_1w_close_001", "FxFwdClose_Otc_All", "GBP", gbpusd_fx_fwd_1w_instr_client_id, maturity_dates['1W'], maturity_dates['1W'], portfolio_otc, units, transaction_price, consideration)
fgn_consideration = 12509
# close out the fwd
close_fwd_request = models.TransactionRequest(
        transaction_id="fx_fwd_1w_close_001",
        #type="FxFwdClose_Otc_All_2",
        type="FxFwdClose_FgnCashSettlementOnly",
        instrument_identifiers={"Instrument/default/ClientInternal": gbpusd_fx_fwd_1w_instr_client_id},
        transaction_date=maturity_dates['1W'].isoformat(),
        settlement_date=maturity_dates['1W'].isoformat(),
        units=units,
        transaction_price=models.TransactionPrice(price=transaction_price, type="Price"),
        total_consideration=models.CurrencyAndAmount(amount=fgn_consideration, currency="USD"),
        exchange_rate=1.2509,
        transaction_currency="GBP",
        properties={
            f"Transaction/{scope}/FgnCcy" : models.PerpetualProperty(
                key=f"Transaction/{scope}/FgnCcy",
                value=models.PropertyValue(label_value="CCY_GBP")
            )
        }
    )

api_factory.build(lusid.api.TransactionPortfoliosApi).upsert_transactions(scope=scope,
                                                                                         code=portfolio_otc,
                                                                                         transaction_request=[close_fwd_request])


{'href': 'https://khalid-local-dev.lusid.com/api/api/transactionportfolios/fx-forward-pricing-nb/fxfwd-portfolio_otc_02/transactions?asAt=2020-07-09T09%3A38%3A50.4088530%2B00%3A00',
 'links': [{'description': None,
            'href': 'https://khalid-local-dev.lusid.com/api/api/portfolios/fx-forward-pricing-nb/fxfwd-portfolio_otc_02?effectiveAt=2020-01-01T00%3A00%3A00.0000000%2B00%3A00&asAt=2020-07-09T09%3A38%3A50.4088530%2B00%3A00',
            'method': 'GET',
            'relation': 'Root'},
           {'description': None,
            'href': 'https://khalid-local-dev.lusid.com/api/api/schemas/entities/UpsertPortfolioTransactionsResponse',
            'method': 'GET',
            'relation': 'EntitySchema'},
           {'description': 'A link to the LUSID Insights website showing all '
                           'logs related to this request',
            'href': 'http://khalid-local-dev.lusid.com/app/insights/logs/0HM13URDH63ML:00000001',
            'method': 'GET',
            '

#### Holdings

In [64]:
holdings_response_trade_date = transaction_portfolios_api.get_holdings(
    scope=scope, code=portfolio_otc, property_keys=["Instrument/default/Name"], effective_at=trade_date
)

trade_date_holdings = display_holdings_summary(holdings_response_trade_date, trade_date)

holdings_response_expiry = transaction_portfolios_api.get_holdings(
    scope=scope, code=portfolio_otc, property_keys=["Instrument/default/Name"], effective_at=maturity_dates['1W']
)

expiry_date_holdings = display_holdings_summary(holdings_response_expiry, maturity_dates['1W'])

pd.concat([trade_date_holdings, expiry_date_holdings])







,names,effectiveAt,units,settled_units,holding types,cost
0,GBP/USD 1W @ 1.2509,2020-06-01 00:00:00+00:00,10000.000000,10000.000000,P,"{'amount': 10000.0, 'currency': 'GBP'}"
0,CCY_GBP,2020-06-08 00:00:00+00:00,-10000.000000,-10000.000000,B,"{'amount': -10000.0, 'currency': 'GBP'}"


In [ ]:
create_transaction_type_configuration(
    api_factory,
    alias=models.TransactionConfigurationTypeAlias(
        type="FxFwdClose_FgnCashSettlementOnly",
        description="Settlment Side of Foreign Cash",
        transaction_class="Basic",
        transaction_group="default",
        transaction_roles="Longer",
    ),
    movements=[
        models.TransactionConfigurationMovementDataRequest(
            movement_types="CashSettlement",
            side="FxFwdFgnCashSide_02",
            direction=1
        )
    ],
)
